In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     #brightness_range=[0.8,1.2], 
                                     shear_range=10)

In [4]:
x1 = train.drop(['id', 'digit', 'letter'], axis=1).values
x1 = x1.reshape(-1, 28, 28, 1)
x1 = x1/255
x1_remake = []
for i in range(x1.shape[0]):
    num_aug = 0
    tmp = x1[i]
    tmp = tmp.reshape((1,) + tmp.shape)
    for x_aug in image_generator.flow(tmp, batch_size = 1) :
        if num_aug >= 1:
            break
        x1_remake.append(x_aug[0])
        num_aug += 1
x1_remake = np.array(x1_remake)

x1_total = np.concatenate((x1, x1_remake), axis=0)
print(x1_total.shape)

(4096, 28, 28, 1)


In [5]:
y1_data = train['digit']
y1 = np.zeros((len(y1_data), len(y1_data.unique())))
for i, digit in enumerate(y1_data):
    y1[i, digit] = 1

y1_remake = y1.copy()
y1_total = np.concatenate((y1, y1_remake), axis=0)
print(y1_total.shape)

(4096, 10)


In [6]:
x1_let = train['letter'].values
x1_let = x1_let[:, np.newaxis]
en = OneHotEncoder()
x1_let = en.fit_transform(x1_let).toarray()

x1_remake_let = x1_let.copy()

x1_letter_total = np.concatenate((x1_let, x1_remake_let), axis=0)
x1_letter_total.shape

(4096, 26)

In [7]:
x1_train, x1_val, y1_train, y1_val = train_test_split(x1_total, y1_total, test_size=0.2, shuffle=True, stratify=y1_total)

print(x1_train.shape)
print(x1_val.shape)
print(y1_train.shape)
print(y1_val.shape)

x1_letter_train = x1_letter_total[:x1_train.shape[0],:]
x1_letter_val = x1_letter_total[x1_train.shape[0]:,:]
print(x1_letter_train.shape)
print(x1_letter_val.shape)

(3276, 28, 28, 1)
(820, 28, 28, 1)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [8]:
x2_train = np.reshape(x1_train, (x1_train.shape[0], x1_train.shape[1], x1_train.shape[2]))
x2_val = np.reshape(x1_val, (x1_val.shape[0], x1_val.shape[1], x1_val.shape[2]))
y2_train = y1_train.copy()
y2_val = y1_val.copy()
x2_letter_train = x1_letter_train.copy()
x2_letter_val = x1_letter_val.copy()

print(x2_train.shape)
print(x2_val.shape)
print(y2_train.shape)
print(y2_val.shape)
print(x2_letter_train.shape)
print(x2_letter_val.shape)

(3276, 28, 28)
(820, 28, 28)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [9]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(input1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge1 = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge1)
x2 = Dropout(0.2)(x2)
x2 = Dense(100, activation='relu')(x2)

input3 = Input(shape=(28,28))
x3 = LSTM(512, activation='relu', return_sequences=True)(input3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(256, activation='relu', return_sequences=True)(x3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(128, activation='relu')(x3)
x3 = Dropout(0.2)(x3)
x3 = Dense(100, activation='relu')(x3)

input4 = Input(shape=(26,))
merge2 = concatenate([x3, input4])

x4 = Dense(100, activation='relu')(merge2)

merge = concatenate([x2, x4])

x5 = Dense(100, activation='relu')(merge)
x5 = Dropout(0.2)(x5)

x6 = Dense(50, activation='relu')(x5)
x6 = Dropout(0.2)(x6)
output1 = Dense(10, activation='softmax')(x6)

x7 = Dense(50, activation='relu')(x5)
output2 = Dense(10, activation='softmax')(x7)

model = Model(inputs = [input1, input2, input3, input4], outputs = [output1, output2])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   320         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (

In [10]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy', 
              loss_weights = [0.8, 0.2])
history = model.fit([x1_train, x1_letter_train, x2_train, x2_letter_train], [y1_train, y2_train],
                    validation_data=([x1_val, x1_letter_val, x2_val, x2_letter_val], [y1_val, y2_val]), 
                    batch_size=64, epochs=50, verbose=1, 
                    callbacks = [ModelCheckpoint('./models/{epoch:02d}-{val_dense_6_acc:.4f}.h5',
                    monitor='val_dense_6_acc', verbose=1, save_best_only=True, mode='auto')])

Train on 3276 samples, validate on 820 samples
Epoch 1/50
3276/3276 [==============================] - 23s 7ms/step - loss: 2.3026 - dense_6_loss: 2.3023 - dense_8_loss: 2.3036 - dense_6_acc: 0.1071 - dense_8_acc: 0.1029 - val_loss: 2.2964 - val_dense_6_loss: 2.2955 - val_dense_8_loss: 2.3002 - val_dense_6_acc: 0.1280 - val_dense_8_acc: 0.1000

Epoch 00001: val_dense_6_acc improved from -inf to 0.12805, saving model to ./models/01-0.1280.h5
Epoch 2/50
3276/3276 [==============================] - 15s 5ms/step - loss: 2.2117 - dense_6_loss: 2.2041 - dense_8_loss: 2.2420 - dense_6_acc: 0.1749 - dense_8_acc: 0.1441 - val_loss: 2.1371 - val_dense_6_loss: 2.1281 - val_dense_8_loss: 2.1733 - val_dense_6_acc: 0.2232 - val_dense_8_acc: 0.1963

Epoch 00002: val_dense_6_acc improved from 0.12805 to 0.22317, saving model to ./models/02-0.2232.h5
Epoch 3/50
3276/3276 [==============================] - 15s 5ms/step - loss: 2.0240 - dense_6_loss: 2.0212 - dense_8_loss: 2.0352 - dense_6_acc: 0.2564 - 


Epoch 00021: val_dense_6_acc improved from 0.70976 to 0.71707, saving model to ./models/21-0.7171.h5
Epoch 22/50
3276/3276 [==============================] - 15s 5ms/step - loss: 0.2919 - dense_6_loss: 0.3001 - dense_8_loss: 0.2591 - dense_6_acc: 0.9023 - dense_8_acc: 0.9148 - val_loss: 0.8763 - val_dense_6_loss: 0.8768 - val_dense_8_loss: 0.8742 - val_dense_6_acc: 0.7061 - val_dense_8_acc: 0.7085

Epoch 00022: val_dense_6_acc did not improve from 0.71707
Epoch 23/50
3276/3276 [==============================] - 15s 5ms/step - loss: 0.2865 - dense_6_loss: 0.2937 - dense_8_loss: 0.2576 - dense_6_acc: 0.9081 - dense_8_acc: 0.9179 - val_loss: 0.9722 - val_dense_6_loss: 0.9728 - val_dense_8_loss: 0.9698 - val_dense_6_acc: 0.6829 - val_dense_8_acc: 0.6866

Epoch 00023: val_dense_6_acc did not improve from 0.71707
Epoch 24/50
3276/3276 [==============================] - 15s 5ms/step - loss: 0.2528 - dense_6_loss: 0.2576 - dense_8_loss: 0.2335 - dense_6_acc: 0.9194 - dense_8_acc: 0.9252 - val

3276/3276 [==============================] - 14s 4ms/step - loss: 0.0793 - dense_6_loss: 0.0815 - dense_8_loss: 0.0707 - dense_6_acc: 0.9762 - dense_8_acc: 0.9789 - val_loss: 1.1060 - val_dense_6_loss: 1.1117 - val_dense_8_loss: 1.0829 - val_dense_6_acc: 0.7146 - val_dense_8_acc: 0.7171

Epoch 00044: val_dense_6_acc did not improve from 0.72683
Epoch 45/50
3276/3276 [==============================] - 15s 4ms/step - loss: 0.0933 - dense_6_loss: 0.0963 - dense_8_loss: 0.0816 - dense_6_acc: 0.9750 - dense_8_acc: 0.9744 - val_loss: 1.1668 - val_dense_6_loss: 1.1714 - val_dense_8_loss: 1.1485 - val_dense_6_acc: 0.6768 - val_dense_8_acc: 0.6720

Epoch 00045: val_dense_6_acc did not improve from 0.72683
Epoch 46/50
3276/3276 [==============================] - 15s 4ms/step - loss: 0.0984 - dense_6_loss: 0.1009 - dense_8_loss: 0.0882 - dense_6_acc: 0.9713 - dense_8_acc: 0.9716 - val_loss: 0.9817 - val_dense_6_loss: 0.9850 - val_dense_8_loss: 0.9687 - val_dense_6_acc: 0.7463 - val_dense_8_acc: 0

In [16]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/46-0.7463.h5')

In [17]:
x1_test = test.drop(['id', 'letter'], axis=1).values
x1_test = x1_test.reshape(-1, 28, 28, 1)
x1_test = x1_test/255

x2_test = test.drop(['id', 'letter'], axis=1).values
x2_test = x2_test.reshape(-1, 28, 28)
x2_test = x2_test/255

x1_letter_test = test['letter'].values
x1_letter_test = x1_letter_test[:, np.newaxis]
en = OneHotEncoder()
x1_letter_test = en.fit_transform(x1_letter_test).toarray()

x2_letter_test = x1_letter_test.copy()

y1_test, y2_test = best_model.predict([x1_test, x1_letter_test, x2_test, x2_letter_test])
y_1 = np.argmax(y1_test, axis=1)
y_2 = np.argmax(y2_test, axis=1)
print(y_1)
print(y_2)

[6 5 2 ... 6 9 0]
[6 0 2 ... 6 9 0]


In [18]:
#submission = pd.read_csv('data/submission.csv')
#submission['digit'] = y_1

In [19]:
#submission.to_csv('data/submission(val7463).csv', index=False)

In [15]:
#submission['digit'] = y_2
#submission.to_csv('data/submission2.csv', index=False)